In [ ]:
# Install libraries
!pip install faiss-cpu sentence-transformers transformers


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving preprocessing_knowledge_base.json to preprocessing_knowledge_base (1).json


In [ ]:
print(uploaded.keys())  # Should display the uploaded file's name


dict_keys(['preprocessing_knowledge_base (1).json'])


In [ ]:
import json

file_path = "preprocessing_knowledge_base (1).json"  # Replace with the correct file name
with open(file_path, "r") as file:
    knowledge_base = json.load(file)

# Verify the structure of the loaded knowledge base
print(knowledge_base.keys())  # Should display the top-level keys


dict_keys(['missing_values', 'outliers', 'scaling', 'encoding', 'domain_specific'])


In [ ]:
documents = []
for category, subcategories in knowledge_base.items():
    for sub_category, techniques in subcategories.items():
        for technique, details in techniques.items():
            if isinstance(details, dict):
                documents.append({
                    "content": details["description"],
                    "category": category,
                    "sub_category": sub_category,
                    "technique": technique,
                    "when_to_use": details.get("when_to_use", "N/A"),
                    "how_to_use": details.get("how_to_use", "N/A"),
                    "limitations": details.get("limitations", "N/A"),
                })

# Verify the flattened structure
print(f"Flattened {len(documents)} documents.")
print(documents[:3])  # Display the first 3 documents for verification


Flattened 8 documents.
[{'content': 'Replace missing numeric values with the mean of the column.', 'category': 'missing_values', 'sub_category': 'numeric', 'technique': 'mean_imputation', 'when_to_use': 'When the data is symmetrically distributed without significant outliers.', 'how_to_use': 'Use tools like SimpleImputer in scikit-learn or pandas fillna().', 'limitations': 'Not suitable for skewed distributions as it can distort data.'}, {'content': 'Replace missing numeric values with the median of the column.', 'category': 'missing_values', 'sub_category': 'numeric', 'technique': 'median_imputation', 'when_to_use': 'When the data is skewed or contains outliers.', 'how_to_use': "Use SimpleImputer with strategy='median' in scikit-learn.", 'limitations': 'May not capture the central tendency for multimodal distributions.'}, {'content': 'Fill missing values using the k-nearest neighbors algorithm.', 'category': 'missing_values', 'sub_category': 'numeric', 'technique': 'knn_imputation', '

In [ ]:
# Extract the "content" field for generating embeddings
texts = [doc["content"] for doc in documents]
print(f"Prepared {len(texts)} documents for embedding.")

Prepared 8 documents for embedding.


In [ ]:
# Step 3: Generate Embeddings for Documents
from sentence_transformers import SentenceTransformer

# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the documents
document_embeddings = embedding_model.encode(texts)
print("Generated embeddings for the knowledge base.")


Generated embeddings for the knowledge base.


In [ ]:
#Create and Save a FAISS Index
import faiss
import numpy as np

# Create FAISS index
dimension = document_embeddings.shape[1]  # Size of embedding vector
index = faiss.IndexFlatL2(dimension)      # Use L2 distance metric
index.add(np.array(document_embeddings))  # Add embeddings to the index

# Save the FAISS index for reuse
faiss.write_index(index, "knowledge_base_index.faiss")
print("FAISS index created and saved.")

# Optional: Download the index
from google.colab import files
files.download("knowledge_base_index.faiss")


FAISS index created and saved.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Query the Knowledge Base
# Encode the query
query = "How should I handle missing values in numeric data?"
query_embedding = embedding_model.encode([query])

# Search the FAISS index
k = 3  # Number of top results
distances, indices = index.search(np.array(query_embedding), k)

# Retrieve the corresponding documents
retrieved_docs = [texts[i] for i in indices[0]]
print("Relevant Knowledge:")
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"{i}. {doc}")

Relevant Knowledge:
1. Use domain-specific knowledge to impute missing values.
2. Replace missing numeric values with the mean of the column.
3. Replace missing numeric values with the median of the column.


In [ ]:
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 req

In [ ]:

!pip install transformers accelerate bitsandbytes auto-gptq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 61.7 MB/s eta 0:00:00


In [ ]:
!pip install ctransformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 20.6 MB/s eta 0:00:00


In [ ]:
!wget https://huggingface.co/TheBloke/Tinyllama-2-1b-miniguanaco-GGUF/resolve/main/tinyllama-2-1b-miniguanaco.Q4_K_M.gguf?download=true -O tinyllama-2-1b-miniguanaco.Q4_K_M.gguf

--2024-11-18 15:02:39--  https://huggingface.co/TheBloke/Tinyllama-2-1b-miniguanaco-GGUF/resolve/main/tinyllama-2-1b-miniguanaco.Q4_K_M.gguf?download=true
Resolving huggingface.co (huggingface.co)... 13.35.210.61, 13.35.210.66, 13.35.210.114, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/f1/9b/f19bdf96e49f3feff6d726558f834c9f3fc8bf8505741527c079d3663b0455b0/56c7638cbc63f03895ac817c6b3d7fba31fc5c8765580cf51dc9e4832cdd56a0?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27tinyllama-2-1b-miniguanaco.Q4_K_M.gguf%3B+filename%3D%22tinyllama-2-1b-miniguanaco.Q4_K_M.gguf%22%3B&Expires=1732201359&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjIwMTM1OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mMS85Yi9mMTliZGY5NmU0OWYzZmVmZjZkNzI2NTU4ZjgzNGM5ZjNmYzhiZjg1MDU3NDE1MjdjMDc5ZDM2NjNiMDQ1NWIwLzU2Yzc2MzhjYm

In [ ]:
#Generate Responses with TinyLLaMA (GGUF)
from ctransformers import AutoModelForCausalLM

# Download GGUF Model
!wget https://huggingface.co/TheBloke/Tinyllama-2-1b-miniguanaco-GGUF/resolve/main/tinyllama-2-1b-miniguanaco.Q4_K_M.gguf -O tinyllama-gguf-model.gguf

# Load the GGUF model
model_path = "tinyllama-gguf-model.gguf"
model = AutoModelForCausalLM.from_pretrained(model_path, model_type="llama")

# Generate response
input_text = f"Question: {query}\n\nRelevant Knowledge:\n" + "\n".join(retrieved_docs)
response = model(input_text, max_new_tokens=300)
print("\nGenerated Response:")
print(response)

--2024-11-19 14:48:00--  https://huggingface.co/TheBloke/Tinyllama-2-1b-miniguanaco-GGUF/resolve/main/tinyllama-2-1b-miniguanaco.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.165.160.59, 3.165.160.11, 3.165.160.12, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/f1/9b/f19bdf96e49f3feff6d726558f834c9f3fc8bf8505741527c079d3663b0455b0/56c7638cbc63f03895ac817c6b3d7fba31fc5c8765580cf51dc9e4832cdd56a0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tinyllama-2-1b-miniguanaco.Q4_K_M.gguf%3B+filename%3D%22tinyllama-2-1b-miniguanaco.Q4_K_M.gguf%22%3B&Expires=1732286880&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjI4Njg4MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mMS85Yi9mMTliZGY5NmU0OWYzZmVmZjZkNzI2NTU4ZjgzNGM5ZjNmYzhiZjg1MDU3NDE1MjdjMDc5ZDM2NjNiMDQ1NWIwLzU2Yzc2MzhjYmM2M2YwMzg5NWFjODE3Y

In [ ]:
# Automate Preprocessing Tasks
import pandas as pd

# Upload and Load Dataset
uploaded_dataset = files.upload()
dataset_path = list(uploaded_dataset.keys())[0]
df = pd.read_csv(dataset_path)

# Automate preprocessing based on LLM response
def apply_preprocessing(df, response):
    if "mean imputation" in response.lower():
        df.fillna(df.mean(), inplace=True)
    elif "drop missing values" in response.lower():
        df.dropna(inplace=True)
    elif "median imputation" in response.lower():
        df.fillna(df.median(), inplace=True)
    return df

df = apply_preprocessing(df, response)
df.to_csv("cleaned_dataset.csv", index=False)
files.download("cleaned_dataset.csv")
print("Preprocessing complete. Cleaned dataset saved.")

Saving fraud_data.csv to fraud_data (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Preprocessing complete. Cleaned dataset saved.


In [ ]:
!zip -r rag-data-preprocessing.zip .


  adding: .config/ (stored 0%)
  adding: .config/config_sentinel (stored 0%)
  adding: .config/default_configs.db (deflated 98%)
  adding: .config/active_config (stored 0%)
  adding: .config/gce (stored 0%)
  adding: .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: .config/.last_update_check.json (deflated 22%)
  adding: .config/.last_survey_prompt.yaml (stored 0%)
  adding: .config/.last_opt_in_prompt.yaml (stored 0%)
  adding: .config/configurations/ (stored 0%)
  adding: .config/configurations/config_default (deflated 15%)
  adding: .config/logs/ (stored 0%)
  adding: .config/logs/2024.11.15/ (stored 0%)
  adding: .config/logs/2024.11.15/14.18.45.909469.log (deflated 93%)
  adding: .config/logs/2024.11.15/14.19.07.543552.log (deflated 58%)
  adding: .config/logs/2024.11.15/14.19.18.989337.log (deflated 85%)
  adding: .config/logs/2024.11.15/14.19.20.453949.log (deflated 58%)
  adding: .config/logs/2024.11.15/14.19.33.079327.log (deflate